# EDA

In [9]:
import pandas as pd

In [1]:
%reload_kedro

2021-04-22 21:42:41,861 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-04-22 21:42:41,915 - root - INFO - ** Kedro project project_student_cup_2021_spr
2021-04-22 21:42:41,917 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-04-22 21:42:41,921 - root - INFO - Registered line magic `run_viz`


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
train_df = catalog.load('train')
train_df.head()

2021-04-22 21:30:36,249 - kedro.io.data_catalog - INFO - Loading data from `train` (CSVDataSet)...


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,genre,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region
0,0,10,11,201094,0.112811,0.157247,0.187841,-1.884852,0.893918,0.363568,0.390108,0.888884,121-152,region_H
1,1,8,69,308493,0.101333,0.346563,0.554444,-5.546495,0.874409,0.193892,0.161497,0.123910,153-176,region_I
2,2,3,43,197225,0.496420,0.265391,0.457642,-9.255670,0.439933,0.217146,0.369057,0.166470,64-76,region_E
3,3,10,45,301092,0.165667,0.245533,0.356578,-5.088788,0.868704,0.377025,0.226677,0.175399,177-192,region_C
4,4,3,57,277348,0.190720,0.777578,0.830479,-3.933896,0.650149,0.169323,0.222488,0.226030,97-120,unknown


In [4]:
test_df = catalog.load('test')
test_df.head()

2021-04-22 21:31:01,026 - kedro.io.data_catalog - INFO - Loading data from `test` (CSVDataSet)...


/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,popularity,duration_ms,acousticness,positiveness,danceability,loudness,energy,liveness,speechiness,instrumentalness,tempo,region
0,4046,43,192465,0.559812,0.565035,0.522237,-12.217087,0.620510,0.389727,0.214989,0.161924,77-96,region_I
1,4047,40,226536,0.134002,0.542547,0.493128,-2.612846,0.853089,0.163106,0.184183,0.155460,121-152,unknown
2,4048,58,215385,0.140537,0.643086,0.533189,-3.439684,0.834416,0.429304,0.167352,0.179191,121-152,region_K
3,4049,49,254494,0.266406,0.456733,0.410583,-10.830424,0.534261,0.151830,0.174403,0.186061,77-96,region_D
4,4050,50,230934,0.351907,0.526235,0.645625,-9.099337,0.612474,0.319161,0.157594,0.184142,97-120,region_E


In [10]:
def one_hot_sklearn(df: pd.DataFrame, drop='first', na_as_category=True) -> pd.DataFrame:
    """
    Notes
    -----
    int型も処理される。必要な列だけ処理されるように注意する。
    """
    if na_as_category:
        df = df.fillna("NA")
        ohe = OneHotEncoder(sparse=False, drop=drop)
        encoded = ohe.fit_transform(df.values)
        label = ohe.get_feature_names(df.columns)
        res = pd.DataFrame(encoded, columns=label, dtype=np.int8)
    else:
        res = pd.DataFrame(index=df.index)
        cat_cols = df.columns
        for c in cat_cols:
            not_null = df[df[c].notnull()][[c]]
            ohe = OneHotEncoder(sparse=False, drop=drop)
            encoded = ohe.fit_transform(not_null.values)
            label = ohe.get_feature_names(not_null.columns)
            res = pd.merge(res, pd.DataFrame(encoded, columns=label, dtype=np.int8, index=not_null.index),
                           left_index=True, right_index=True, how='left')

    return res